In [1]:
from downloader import ENCODEDownloader
from pybedtools import BedTool
import pandas as pd
import gffutils
from processor import ENCODEProcessor
import re

In [2]:
downloader = ENCODEDownloader()
# downloader.download_reference_genome()
# downloader.download_experiments(assay="ATAC-seq", n_experiments=2)

In [3]:
gene_db = downloader.get_gene_db(
    db_file="/home/tjkatz/genbio_hw/encode_data/genomes/GRCh38_v24_annotation.sqlite"
)

Loading existing database from /home/tjkatz/genbio_hw/encode_data/genomes/GRCh38_v24_annotation.sqlite...
Database loaded successfully


In [4]:
processor = ENCODEProcessor(
    output_dir="encode_data",
    db_file="/home/tjkatz/genbio_hw/encode_data/genomes/GRCh38_v24_annotation.sqlite",
)

Loading existing database from /home/tjkatz/genbio_hw/encode_data/genomes/GRCh38_v24_annotation.sqlite...
Database loaded successfully


In [5]:
t_to_g, g_to_t = processor.get_transcript_gene_mapping()

In [6]:
len(t_to_g), len(g_to_t)

(143984, 19844)

In [7]:
i = 0
for gene_id, transcript_ids in g_to_t.items():
    print(gene_id, transcript_ids)
    i += 1
    if i > 4:
        break

ENSG00000186092.4 ['ENST00000335137.3']
ENSG00000279928.1 ['ENST00000624431.1']
ENSG00000279457.3 ['ENST00000623834.3', 'ENST00000623083.3', 'ENST00000624735.1']
ENSG00000278566.1 ['ENST00000426406.2']
ENSG00000273547.1 ['ENST00000332831.3']


In [8]:
genes_bed = BedTool(
    [
        (gene.seqid, gene.start - 1, gene.end, gene.id, 0, gene.strand)
        for gene in processor.gene_db.features_of_type("gene")
    ]
)

In [9]:
atac_peaks = BedTool(
    "/home/tjkatz/genbio_hw/encode_data/experiments/ATAC-seq/ENCSR504OUW/ENCFF578ODL.bed"
)
extended_peaks = atac_peaks.slop(b=10000, genome="hg38")
merged_peaks = extended_peaks.sort().merge()

In [10]:
peak_gene_intersect = merged_peaks.intersect(genes_bed, wo=True)

In [11]:
peak_gene_intersect.head()

chr1	619075	644615	chr1	629061	629433	ENSG00000225972.1	0	+	372
 chr1	619075	644615	chr1	629639	630683	ENSG00000225630.1	0	+	1044
 chr1	619075	644615	chr1	631073	632616	ENSG00000237973.1	0	+	1543
 chr1	619075	644615	chr1	632756	633438	ENSG00000229344.1	0	+	682
 chr1	619075	644615	chr1	633534	633741	ENSG00000240409.1	0	+	207
 chr1	619075	644615	chr1	633695	634376	ENSG00000248527.1	0	+	681
 chr1	619075	644615	chr1	634375	634922	ENSG00000198744.5	0	+	547
 chr1	619075	644615	chr1	585988	827796	ENSG00000230021.8	0	-	25540
 chr1	768512	789132	chr1	725884	778626	ENSG00000228327.3	0	-	10114
 chr1	768512	789132	chr1	585988	827796	ENSG00000230021.8	0	-	20620
 

In [12]:
len(peak_gene_intersect)

41022

In [13]:
gene_ids = set()
for peak in peak_gene_intersect:
    gene_ids.add(peak[6])
print(len(gene_ids))

30080


In [14]:
trans_ids = set()
for gene_id in gene_ids:
    trans_ids.update(g_to_t[gene_id])
print(len(trans_ids))

132788


In [15]:
test_ids = list(trans_ids)[:50]
results = processor.map_transcripts_to_uniprot(transcript_ids=test_ids)

Retrying in 3s
Fetched: 30 / 30


In [16]:
results

{'ENST00000368029': 'Q8WWU7',
 'ENST00000393452': 'E7EPD8',
 'ENST00000498214': 'A8K3Q5',
 'ENST00000181383': 'Q96IY4',
 'ENST00000619918': 'Q9UFD9',
 'ENST00000372217': 'Q99661',
 'ENST00000568378': 'H3BMR9',
 'ENST00000504897': 'G3V1J0',
 'ENST00000598249': 'Q9H254',
 'ENST00000539865': 'F5H023',
 'ENST00000455757': 'H7C1R5',
 'ENST00000435470': 'O95278',
 'ENST00000367449': 'Q9Y5K5',
 'ENST00000622874': 'E9JVC4',
 'ENST00000547579': 'F8W1F2',
 'ENST00000345378': 'P35557',
 'ENST00000471098': 'C9IZ32',
 'ENST00000333991': 'O14727',
 'ENST00000301264': 'O43293',
 'ENST00000308423': 'Q16853',
 'ENST00000541435': 'Q96DB9',
 'ENST00000568312': 'I3L306',
 'ENST00000542869': 'Q13882',
 'ENST00000481367': 'H7C5G1',
 'ENST00000430527': 'E9PRN3',
 'ENST00000529749': 'E9PIG6',
 'ENST00000361092': 'Q9NWW6',
 'ENST00000361490': 'Q92902',
 'ENST00000355149': 'Q8TCZ2',
 'ENST00000392746': 'F8W689'}